In [2]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
import os
from pprint import pprint
import json
import urllib2
import xml.etree.ElementTree as ET
import csv
import pandas as pd
from datetime import datetime, timedelta
import pymongo

In [13]:
f = urllib2.urlopen('http://api.wunderground.com/api/10f51b471553d661/geolookup/conditions/q/IA/Kuopio.json')
json_string = f.read()
parsed_json = json.loads(json_string)
location = parsed_json['location']['city']
temp_c = parsed_json['current_observation']['temp_c']
print "Current temperature in %s is: %s" %(location, temp_c)
f.close

Current temperature in Kuopio is: 7.6


<bound method addinfourl.close of <addinfourl at 4519452184 whose fp = <socket._fileobject object at 0x10cda18d0>>>

In [199]:
r = requests.get('http://data.fmi.fi/fmi-apikey/25b57e29-e541-4ce1-8645-fb5e4e47ed40/wfs?request=getFeature&storedquery_id=fmi::observations::weather::simple&place=kumpula,Helsinki&timestep=30&parameters=temperature&starttime=2010-01-01T00:00:00Z')
xmlstring = r.content

root = ET.fromstring(xmlstring)

outfile = open("temperature.csv", "w")
writer = csv.writer(outfile)

header = ("timestamp", "temperature")
writer.writerow(header)

for child in root.iter('{http://xml.fmi.fi/schema/wfs/2.0}BsWfsElement'):
    parameter_name = child.find('{http://xml.fmi.fi/schema/wfs/2.0}ParameterName').text
    if parameter_name != "temperature":
        continue
    
    parameter_value = child.find('{http://xml.fmi.fi/schema/wfs/2.0}ParameterValue').text
    timestamp = child.find('{http://xml.fmi.fi/schema/wfs/2.0}Time').text
    row = (timestamp, parameter_value)
    writer.writerow(row)
    
outfile.close()

In [3]:

df = pd.read_csv("temperature.csv")

last_ts = (df['timestamp'].iloc[-1])

ts = datetime.strptime(last_ts, "%Y-%m-%dT%H:%M:%SZ")
ts += timedelta(minutes=30)

string_ts = datetime.strftime(ts, "%Y-%m-%dT%H:%M:%SZ")

starttime = '&starttime=' + string_ts

api_key = '25b57e29-e541-4ce1-8645-fb5e4e47ed40'
url = 'http://data.fmi.fi/fmi-apikey/' + api_key + '/wfs?request=getFeature&storedquery_id=fmi::observations::weather::simple&place=kumpula,Helsinki' + starttime + '&timestep=30&parameters=temperature'

running_r = requests.get(url)
running_s = running_r.content

r_root = ET.fromstring(running_s)

updatefile = open("temperature.csv", "a")
update_writer = csv.writer(updatefile)

for child in r_root.iter('{http://xml.fmi.fi/schema/wfs/2.0}BsWfsElement'):
    parameter_name = child.find('{http://xml.fmi.fi/schema/wfs/2.0}ParameterName').text
    if parameter_name != "temperature":
        continue
    
    parameter_value = child.find('{http://xml.fmi.fi/schema/wfs/2.0}ParameterValue').text
    timestamp = child.find('{http://xml.fmi.fi/schema/wfs/2.0}Time').text
    row = (timestamp, parameter_value)
    update_writer.writerow(row)
    
updatefile.close()

In [6]:
# Setting starting date and time for FMI Rest api call
startts = "2010-01-01T00:00:00Z"

# Creates variables for looping and 7-days periods and months
# FMI uses 7 days period for 30min measures and that is why we need to use these variables
starttime = datetime.strptime(startts, "%Y-%m-%dT%H:%M:%SZ")
next_month = datetime(starttime.year + (starttime.month / 12), ((starttime.month % 12) + 1), 1)
# Stopdate is user defined variable. This is a last month that will be fetched
stopdate = datetime.strptime("2010-02-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ")
runningweek = starttime
continuedate = starttime

# Loop structure for REST api call - first while only checks if stopdate is reached
# second while creates 7 days rest api call for temperatures
# Check for is running date is past stopdate
while (continuedate <= stopdate):
    
    # Naming file name for saving data: year-month-first day
    savename = datetime(runningweek.year, runningweek.month, 1)
    filename = datetime.strftime(savename, "%Y-%m-%d")
    
    # Open file for writing data
    f = open(filename + ".csv", "w")
    writer = csv.writer(f)
    
    # Writing header to csv-file
    header = ("timestamp", "temperature")
    writer.writerow(header)
    
    # Checking if month has changed, if so closing file and save next month number
    while(runningweek <= next_month):
        
        # creating starttime for url
        start_string = datetime.strftime(runningweek, "%Y-%m-%dT%H:%M:%SZ")
        starttime = '&starttime=' + start_string
        
        # creating endtime for url
        ending = runningweek + timedelta(days=7)
        
        # Checking if ending date is bigger than next_month, if so taking next_month datetime minus 30 mins
        # this will prevent over writing from monthly data
        if(ending > next_month):
            ending = next_month - timedelta(minutes=30)
            end_string = datetime.strftime(ending, "%Y-%m-%dT%H:%M:%SZ")
            endtime = '&endtime=' + end_string
        else:
            end_string = datetime.strftime(ending, "%Y-%m-%dT%H:%M:%SZ")
            endtime = '&endtime=' + end_string
                
        # creating rest api url
        api_key = '25b57e29-e541-4ce1-8645-fb5e4e47ed40'
        url = 'http://data.fmi.fi/fmi-apikey/' + api_key + '/wfs?request=getFeature&storedquery_id=fmi::observations::weather::simple&place=kumpula,Helsinki' + starttime + endtime + '&timestep=30&parameters=temperature'

        # sending requests and saving content from it
        running_r = requests.get(url)
        running_s = running_r.content

        r_root = ET.fromstring(running_s)
            

        #updatefile = open("temperature.csv", "a")
        update_writer = csv.writer(f)
        
        # iterating xml data and finding temperature measure parameter
        for child in r_root.iter('{http://xml.fmi.fi/schema/wfs/2.0}BsWfsElement'):
            parameter_name = child.find('{http://xml.fmi.fi/schema/wfs/2.0}ParameterName').text
            if parameter_name != "temperature":
                continue
        # writing timestamp and parameter_value to file
            parameter_value = child.find('{http://xml.fmi.fi/schema/wfs/2.0}ParameterValue').text
            timestamp = child.find('{http://xml.fmi.fi/schema/wfs/2.0}Time').text
            #print "%s;%s" % (timestamp, parameter_value)
            row = (timestamp, parameter_value)
            update_writer.writerow(row)
    
        # adding new startdate for loop 7 days + 30min
        runningweek = runningweek + timedelta(days=7) + timedelta(minutes=30)
        
    else:
        runningweek = next_month
        next_month = datetime(next_month.year + (next_month.month / 12), ((next_month.month % 12) + 1), 1)
        continuedate = runningweek
        f.close()
else :
    print("loop completed")

# just clearing variables for sure
next_month = None
runningweek = None
continuedate = None

loop completed


In [3]:
client=pymongo.MongoClient('localhost',27017)
db=client.HelWeather
serverStatusResult = db.command("serverStatus")
print(serverStatusResult)

{u'process': u'mongod', u'pid': 97254L, u'connections': {u'current': 3, u'available': 5731, u'totalCreated': 5}, u'locks': {u'Metadata': {u'acquireCount': {u'w': 1L}}, u'Global': {u'acquireCount': {u'r': 1768L, u'W': 3L, u'w': 8L}}, u'Collection': {u'acquireCount': {u'r': 618L}}, u'Database': {u'acquireCount': {u'R': 7L, u'r': 626L, u'W': 8L}}}, u'storageEngine': {u'readOnly': False, u'supportsCommittedReads': True, u'name': u'wiredTiger', u'persistent': True}, u'globalLock': {u'totalTime': 489569000L, u'currentQueue': {u'total': 0, u'writers': 0, u'readers': 0}, u'activeClients': {u'total': 9, u'writers': 0, u'readers': 0}}, u'extra_info': {u'note': u'fields vary by platform', u'page_faults': 174}, u'uptimeEstimate': 489L, u'uptime': 489.0, u'network': {u'physicalBytesOut': 9958L, u'numRequests': 73L, u'bytesOut': 9958L, u'physicalBytesIn': 3419L, u'bytesIn': 3419L}, u'uptimeMillis': 489579L, u'version': u'3.4.9', u'mem': {u'resident': 20, u'supported': True, u'virtual': 3170, u'mappe

In [10]:
weather = db.HelWeather
temp_data = {
    'ts': [datetime.strptime('2017-11-01', '%Y-%m-%d'), datetime.strptime('2017-11-02', '%Y-%m-%d'), datetime.strptime('2017-11-03', '%Y-%m-%d')],
    'temperature': [5,6,4]
}

result = weather.insert_many(temp_data)

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

In [34]:
# This will create list of year-months csv-files. startts is starting date and stopdate is the last month to create

startts = "2010-01-01T00:00:00Z"

next_month = datetime.strptime(startts, "%Y-%m-%dT%H:%M:%SZ")
stopdate = datetime.strptime("2012-01-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ")
month_list =[]

while (next_month <= stopdate):
    month_list.append(next_month.strftime('%Y-%m-%d'))
    next_month = datetime(next_month.year + (next_month.month / 12), ((next_month.month % 12) + 1), 1)


for item in month_list:
    f = open(item + ".csv", "w")
    f.close()

month_list = []

In [7]:
# Setting starting date and time for FMI Rest api call
startts = "2010-01-01T00:00:00Z"

# Creates variables for looping and 7-days periods and months
# FMI uses 7 days period for 30min measures and that is why we need to use these variables
starttime = datetime.strptime(startts, "%Y-%m-%dT%H:%M:%SZ")
next_month = datetime(starttime.year + (starttime.month / 12), ((starttime.month % 12) + 1), 1)
# Stopdate is user defined variable. This is a last month that will be fetched
stopdate = datetime.strptime("2010-02-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ")
runningweek = starttime
continuedate = starttime

# Loop structure for REST api call - first while only checks if stopdate is reached
# second while creates 7 days rest api call for temperatures
# Check for is running date is past stopdate
while (continuedate <= stopdate):
    
    # Naming file name for saving data: year-month-first day
    savename = datetime(runningweek.year, runningweek.month, 1)
    filename = datetime.strftime(savename, "%Y-%m-%d")
    
    # Open file for writing data
    #f = open(filename + ".csv", "w")
    #writer = csv.writer(f)
    
    # Writing header to csv-file
    #header = ("timestamp", "temperature")
    #writer.writerow(header)
    
    # Checking if month has changed, if so closing file and save next month number
    while(runningweek <= next_month):
        
        # creating starttime for url
        start_string = datetime.strftime(runningweek, "%Y-%m-%dT%H:%M:%SZ")
        starttime = '&starttime=' + start_string
        
        # creating endtime for url
        ending = runningweek + timedelta(days=7)
        
        # Checking if ending date is bigger than next_month, if so taking next_month datetime minus 30 mins
        # this will prevent over writing from monthly data
        if(ending > next_month):
            ending = next_month - timedelta(minutes=30)
            end_string = datetime.strftime(ending, "%Y-%m-%dT%H:%M:%SZ")
            endtime = '&endtime=' + end_string
        else:
            end_string = datetime.strftime(ending, "%Y-%m-%dT%H:%M:%SZ")
            endtime = '&endtime=' + end_string
                
        # creating rest api url
        api_key = '25b57e29-e541-4ce1-8645-fb5e4e47ed40'
        url = 'http://data.fmi.fi/fmi-apikey/' + api_key + '/wfs?request=getFeature&storedquery_id=fmi::observations::weather::simple&place=kumpula,Helsinki' + starttime + endtime + '&timestep=30&parameters=temperature,humidity,winddirection,windspeedms'

        # sending requests and saving content from it
        running_r = requests.get(url)
        running_s = running_r.content

        r_root = ET.fromstring(running_s)
            

        #updatefile = open("temperature.csv", "a")
        #update_writer = csv.writer(f)
        
        measure = {}
        new_ts = starttime
        
        # iterating xml data and finding temperature measure parameter
        for child in r_root.iter('{http://xml.fmi.fi/schema/wfs/2.0}BsWfsElement'):
            parameter_name = str(child.find('{http://xml.fmi.fi/schema/wfs/2.0}ParameterName').text)
        #    if parameter_name != "temperature":
        #        continue
        # writing timestamp and parameter_value to file
            parameter_value = child.find('{http://xml.fmi.fi/schema/wfs/2.0}ParameterValue').text
            timestamp = child.find('{http://xml.fmi.fi/schema/wfs/2.0}Time').text
            measure['timestamp'] = timestamp
            measure['parameter_name'] = parameter_name
            measure['parameter_value'] = parameter_value
            
            print(measure)
            #print "%s;%s;%s" % (timestamp, parameter_name, parameter_value)
            
            #row = (timestamp, parameter_value)
            #update_writer.writerow(row)
    
        # adding new startdate for loop 7 days + 30min
        runningweek = runningweek + timedelta(days=7) + timedelta(minutes=30)
        
    else:
        runningweek = next_month
        next_month = datetime(next_month.year + (next_month.month / 12), ((next_month.month % 12) + 1), 1)
        continuedate = runningweek
        f.close()
else :
    print("loop completed")

# just clearing variables for sure
next_month = None
runningweek = None
continuedate = None

{'timestamp': '2010-01-01T00:00:00Z', 'parameter_value': '-10.6', 'parameter_name': 'temperature'}
{'timestamp': '2010-01-01T00:00:00Z', 'parameter_value': '88.0', 'parameter_name': 'humidity'}
{'timestamp': '2010-01-01T00:00:00Z', 'parameter_value': '41.0', 'parameter_name': 'winddirection'}
{'timestamp': '2010-01-01T00:00:00Z', 'parameter_value': '5.2', 'parameter_name': 'windspeedms'}
{'timestamp': '2010-01-01T00:30:00Z', 'parameter_value': '-10.6', 'parameter_name': 'temperature'}
{'timestamp': '2010-01-01T00:30:00Z', 'parameter_value': '89.0', 'parameter_name': 'humidity'}
{'timestamp': '2010-01-01T00:30:00Z', 'parameter_value': '42.0', 'parameter_name': 'winddirection'}
{'timestamp': '2010-01-01T00:30:00Z', 'parameter_value': '5.6', 'parameter_name': 'windspeedms'}
{'timestamp': '2010-01-01T01:00:00Z', 'parameter_value': '-10.6', 'parameter_name': 'temperature'}
{'timestamp': '2010-01-01T01:00:00Z', 'parameter_value': '88.0', 'parameter_name': 'humidity'}
{'timestamp': '2010-01-0

In [13]:
keys = ['ensimmäinen', 'toinen', 'kolmas']
values = [12.6, 23, 4, 0.6]

measure = {}
measure = {k:v for k, v in zip(keys, values)}
print(measure)

{'kolmas': 4, 'ensimm\xc3\xa4inen': 12.6, 'toinen': 23}
